## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 장르 추천

### 세부 장르 가져오기

In [2]:
# 파일에서 세부 장르 데이터 가져오기

open_file = open('./text files/detail_genre.txt', 'r', encoding = 'utf8')
text = open_file.read()
open_file.close()

# 파일에서 읽어온 데이터 확인
print(text)

data_list = text.split('\n')

# data_list 의 마지막에 공백이 추가되므로 공백 제거
data_list = data_list[:-2]
# 확인
print(data_list)

TV 시사&교양-기타
TV 시사&교양-인물&다큐
TV 연예&오락-기타
TV드라마-기타
TV드라마-외화 시리즈
TV애니메이션-기타
TV애니메이션-명랑&코믹
TV애니메이션-액션&모험
TV애니메이션-추리&미스터리
TV애니메이션-학원&순정&연애
다큐-인물
영화-SF&환타지
영화-공포&스릴러
영화-다큐멘터리
영화-단편
영화-드라마
영화-로맨틱코미디
영화-멜로
영화-무협
영화-애니메이션
영화-액션&어드벤쳐
영화-역사
영화-코미디
우리동네-연예&오락
키즈-기타
키즈-애니메이션
키즈-오락
키즈-학습

['TV 시사&교양-기타', 'TV 시사&교양-인물&다큐', 'TV 연예&오락-기타', 'TV드라마-기타', 'TV드라마-외화 시리즈', 'TV애니메이션-기타', 'TV애니메이션-명랑&코믹', 'TV애니메이션-액션&모험', 'TV애니메이션-추리&미스터리', 'TV애니메이션-학원&순정&연애', '다큐-인물', '영화-SF&환타지', '영화-공포&스릴러', '영화-다큐멘터리', '영화-단편', '영화-드라마', '영화-로맨틱코미디', '영화-멜로', '영화-무협', '영화-애니메이션', '영화-액션&어드벤쳐', '영화-역사', '영화-코미디', '우리동네-연예&오락', '키즈-기타', '키즈-애니메이션', '키즈-오락']


### 불용어

In [39]:
with open('./text files/stopword.txt', 'r', encoding = 'utf-8') as f:
    stopwords = f.read()

#print(type(stopwords))
stopwords_list = stopwords.split('\n')
#print(stopwords_list)

### 세부 장르별 단어 가져오기

In [14]:
from konlpy.tag import Twitter
import nltk

In [165]:
word_dict_data = {}

# 각 세부 장르 별로 단어 리스트 생성하고 저장
for detail_genre in data_list:
    # 각 세부 장르에 대해 단어 데이터 가져오기
    open_file = open('./text files/' + detail_genre + '_words.txt', 'r', encoding = 'utf8')
    text = open_file.read()
    
    # 파일 닫기
    open_file.close()
    
    # 각 단어 분리하기
    text_list = text.split('\n')
    text_list = text_list[:-2] # 마지막의 공백 제외
    
    # 각 세부 장르별 줄거리의 단어 갯수 확인 => 가장 단어의 갯수가 적은 경우는 15개
    #no_dup = set(text)
    #print(detail_genre, ' : ', len(text), len(no_dup))
    
    # 각 장르별 주요 단어 찾기
    spliter = Twitter()
    nouns_for_dictionary = spliter.nouns(text)
    ko = nltk.Text(nouns_for_dictionary, name = detail_genre)
    
    
    # 불용어 제거
    ko1 = [each_word for each_word in ko if each_word not in stopwords_list]
    ko = nltk.Text(ko1, name = detail_genre)
    word_dict = dict(ko.vocab())

    # 확인
    #print(type(word_dict))
    sorted_dictionary = sorted(word_dict.items(), key = lambda item: item[1], reverse = True)
    #print(sorted_dictionary)
    
    # 단어 리스트를 dict 형태로 저장
    word_dict_data[detail_genre] = word_dict

In [78]:
print(stopwords_list[0], stopwords_list[6])
print(type(stopwords_list[0]))

않다 이다
<class 'str'>


In [79]:
word_dict_data['영화-역사']

{'땅': 1,
 '기운': 1,
 '점': 1,
 '인간': 1,
 '운명': 1,
 '수': 1,
 '천재': 1,
 '지관': 1,
 '박재상': 1,
 '명당': 1,
 '이용': 1,
 '천하': 1,
 '권력': 1,
 '장동': 1,
 '김씨': 1,
 '가문': 1,
 '계획': 1,
 '가족': 1,
 '후': 1,
 '복수': 1,
 '왕족': 1,
 '흥선': 1}

### 세부 장르별 단어 정리

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [81]:
# 각 장르별 단어 저장하기

# 장르별 벡터 저장
word_vector_dict = {}

okt = Okt()
vectorizer = CountVectorizer(min_df = 0.05)
    
# dict 형태로 저장되어 있는 단어 데이터를 읽고
# 각 단어들을 모아서 하나의 문장으로 만든 다음 저장
for detail_genre in data_list:
    contents_tokens = word_dict_data[detail_genre]

    # 벡터화를 위해 단어들을 가지고 문장 생성
    contents_for_vect = []
    sentence = ''
    # 토큰 단위로 구분된 문장을 생성
    for content in contents_tokens:
        sentence += ' ' + content

    # 생성한 문장을 리스트에 추가
    contents_for_vect.append(sentence)
    
    word_vector_dict[detail_genre] = contents_for_vect

In [164]:
#word_vector_dict['TV 시사&교양-기타'][0]

In [83]:
contents_for_vect

[' 아이스크림 모기 탭댄스 버섯 껌 빙판 빨대 식충식물 달팽이 코골 팝콘 수족관 햄 초능력 파리 스파게티']

## 샘플 줄거리 데이터를 활용해서 거리 계산

In [84]:
from konlpy.tag import Twitter

In [149]:
# 샘플 데이터와 훈련 데이터 거리 확인

import scipy as sp

#거리 구해주는 함수 생성
def dist_raw(v1, v2):
    # 차이를 계산
    delta = v1 - v2
    
    return sp.linalg.norm(delta.toarray())

In [121]:
import math

In [163]:
# 샘플 줄거리 데이터 
new_content = ' 밀실 안의 살인자, 정유정은 누구인가? 20대 또래 여성을 살해한 정유정의 정체와 범행 동기가 무엇인지에 대해 추적해 본다.'

# 샘플 문장 토큰화
spliter = Twitter()
sample_words = spliter.nouns(new_content)
del_sample_words = ['정유', '정은', '누구', '정의', '무엇', '인지', '대해']
sample_words = [each_word for each_word in sample_words if each_word not in del_sample_words]

# 가장 거리가 짧은 세부 장르 계산용
min_distance = 65536
min_detail_genre = 'None'

# 각 세부 장르별 줄거리를 가지고 단어 사전을 생성하고 샘플 줄거리와
# 거리를 비교
for detail_genre in data_list:
    vectorizer = CountVectorizer(min_df = 1) # 1번만 등장해도 단어 사전에 포함시키도록
    
    # 장르 별로 줄거리 불러오기 - dict 의 value
    contents_tokens = word_vector_dict[detail_genre]
    
    # 문장 생성
    sample_sentence = ''
    for sample_word in sample_words:
        sample_sentence += (sample_word + ' ')
        
    sentence = contents_tokens[0] #+ sample_sentence
    #print(sentence + '\n')
    
    # 생성한 문장을 리스트에 추가
    contents_for_vect = []
    contents_for_vect.append(sentence)
    
    # 피처 벡터화 - 띄어쓰기를 기준으로 벡터화
    X = vectorizer.fit_transform(contents_for_vect)
    # 피처 확인
    #print(vectorizer.get_feature_names_out())
    
    # 샘플 줄거리 문장을 피처 벡터화
    new_content_vect = vectorizer.transform([sample_sentence])
    #print(new_content_vect)
    
    # 거리 계산
    post_vec = X.getrow(0) # 문장이 1개이므로 첫번째인 0번 인덱스
    #print(post_vec)
    #print(post_vec.shape[1])
    #print(new_content_vect.shape[1])
    distance = dist_raw(new_content_vect, post_vec)
    
    # 세부 장르별 단어의 갯수를 세고 일정 갯수 이하이면 제외
    length = post_vec.shape[1]
    limit_length = 100
    limit_cor = 0.3
    if length < limit_length:
        pass
    
    else:
        # 일치율 계산하기
        count = 0
        for item in sample_words:
            if item in contents_tokens[0]:
                count += 1

        cor = count / len(sample_words)
        
        # 일치율이 기준을 넘는 경우에만 거리를 계산
        if cor < limit_cor:
            pass
        
        else:
            # 결과 확인 - 장르와의 거리는 줄거리 단어의 길이를 가중치로 반영하여 계산
            weighted_distance = distance / (math.log10(length)) / length * 1000
            print(detail_genre, ' 장르와의 거리는 : \t', weighted_distance, sep='') 
            # 1000을 곱한 이유는 쉽게 보기 위함
            print(detail_genre, ' 장르와의 거리(원본)는 : \t', distance, sep='')
            print(detail_genre, ' 장르와의 일치율은 : \t', cor * 100 , '%', sep='')
            print(detail_genre, ' 장르의 단어 길이는 : \t', length, '\n',sep='')
            
            if weighted_distance < min_distance:
                min_distance = weighted_distance
                min_detail_genre = detail_genre

# 최종 결과         
print('가장 유사한 장르는 ', min_detail_genre, sep = '')

TV 시사&교양-기타 장르와의 거리는 : 	8.587169321076978
TV 시사&교양-기타 장르와의 거리(원본)는 : 	36.932370625238775
TV 시사&교양-기타 장르와의 일치율은 : 	80.0%
TV 시사&교양-기타 장르의 단어 길이는 : 	1371

TV 연예&오락-기타 장르와의 거리는 : 	4.438721461438541
TV 연예&오락-기타 장르와의 거리(원본)는 : 	62.5939294181153
TV 연예&오락-기타 장르와의 일치율은 : 	70.0%
TV 연예&오락-기타 장르의 단어 길이는 : 	3924

TV드라마-기타 장르와의 거리는 : 	6.06106949245715
TV드라마-기타 장르와의 거리(원본)는 : 	48.75448697299562
TV드라마-기타 장르와의 일치율은 : 	60.0%
TV드라마-기타 장르의 단어 길이는 : 	2382

TV드라마-외화 시리즈 장르와의 거리는 : 	4.837502111176168
TV드라마-외화 시리즈 장르와의 거리(원본)는 : 	58.42944463196617
TV드라마-외화 시리즈 장르와의 일치율은 : 	50.0%
TV드라마-외화 시리즈 장르의 단어 길이는 : 	3418

영화-공포&스릴러 장르와의 거리는 : 	19.508777898098554
영화-공포&스릴러 장르와의 거리(원본)는 : 	19.595917942265423
영화-공포&스릴러 장르와의 일치율은 : 	50.0%
영화-공포&스릴러 장르의 단어 길이는 : 	388

영화-드라마 장르와의 거리는 : 	11.878666268561824
영화-드라마 장르와의 거리(원본)는 : 	28.74021572639983
영화-드라마 장르와의 일치율은 : 	40.0%
영화-드라마 장르의 단어 길이는 : 	829

영화-멜로 장르와의 거리는 : 	19.104041956023245
영화-멜로 장르와의 거리(원본)는 : 	20.0
영화-멜로 장르와의 일치율은 : 	30.0%
영화-멜로 장르의 단어 길이는 : 	402

영화-애니메이션 장르와의 거리